# Analysis of Density Functional Theory on a 1d Ising Model with 1nn interaction

The Hamiltonian is given by

$H=J \sum_i x_i x_{i+1} + \sum_i h_i z_i $  $\; \;$ with $h_i \in [0,h_{max}]$ and $J<0$

## Case a) Scalability of the Functional at different $h_{max}$

#### Imports

In [ ]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from scipy import fft, ifft
from tqdm.notebook import tqdm, trange
from src.training.utils_analysis import dataloader, nuv_representability_check

#### Data

In [ ]:
n_sample=7
t_range=[49,49,49,199,199,799,799]
h_max=[4.5]*n_sample
ls=[16,32,64,128,256,512,1024]
n_instances=[100]*n_sample
epochs = [[i * 100 for i in range(t_range[j])] for j in range(n_sample)] 
# model settings
name_session=[f'h_{h_max[i]}_150k_augmentation_1nn_model_unet_{ls[i]}_size_2_layers_20_hc_5_ks_2_ps' for i in range(n_sample)]
early_stopping=False
variational_lr=False
loglr=1

min_density={}
gs_density={}
min_energy={}
gs_energy={}

for i in range(n_sample):
    min_eng=[]
    gs_eng=[]
    min_n=[]
    gs_n=[]
    for j in range(t_range[i]):    
        min_eng_t,gs_eng_t=dataloader('energy',session_name=name_session[i],n_instances=n_instances[i],lr=loglr,diff_soglia=1,epochs=epochs[i][j],early_stopping=False,variable_lr=False,n_ensambles=1)
        min_n_t,gs_n_t=dataloader('density',session_name=name_session[i],n_instances=n_instances[i],lr=loglr,diff_soglia=1,epochs=epochs[i][j],early_stopping=False,variable_lr=False,n_ensambles=1)
        
        min_eng_t=np.asarray(min_eng_t)
        gs_eng_t=np.asarray(gs_eng_t)
        min_n_t=np.asarray(min_n_t)
        gs_n_t=np.asarray(gs_n_t)
        
        if j==0:
            min_eng=min_eng_t.reshape(1,-1)
            gs_eng=gs_eng_t.reshape(1,-1)
            min_n=min_n_t.reshape(1,-1,ls[i])
            gs_n=gs_n_t.reshape(1,-1,ls[i])
        else:
            #if min_eng_t.shape[0]==min_eng.shape[-1]:
            min_eng=np.append(min_eng,min_eng_t.reshape(1,-1),axis=0)
            gs_eng=np.append(gs_eng,gs_eng_t.reshape(1,-1),axis=0)
            min_n=np.append(min_n,min_n_t.reshape(1,-1,ls[i]),axis=0)
            gs_n=np.append(gs_n,gs_n_t.reshape(1,-1,ls[i]),axis=0)
            
        
    # min_eng=np.asarray(min_eng,dtype=object)
    # gs_eng=np.asarray(gs_eng,dtype=object)
    # min_n=np.asarray(min_n,dtype=object)
    # gs_n=np.asarray(gs_n,dtype=object)

    min_energy[ls[i]]=min_eng
    gs_energy[ls[i]]=gs_eng
    min_density[ls[i]]=min_n
    gs_density[ls[i]]=gs_n
    print(gs_n.shape)

#### Convergence of $|\Delta e|/|e|$ for different sizes

In [ ]:
fig=plt.figure(figsize=(10,10))
errors_e=[]
sigma_errors_e=[]
for l in ls:
    print(l)
    print(min_energy[l].shape)
    print(gs_energy[l].shape)
    e_av=np.average(np.abs(min_energy[l]-gs_energy[l])/np.abs(gs_energy[l]),axis=-1)
    plt.plot(e_av,label=f'l={l}',linewidth=3)
    errors_e.append(e_av[-1])
    sigma_errors_e.append(np.std(np.abs(min_energy[l]-gs_energy[l])/np.abs(gs_energy[l]),axis=-1)[-1])
plt.legend(fontsize=20)
plt.xlabel(r'$t$',fontsize=40)
plt.ylabel(r'$\langle\frac{\left| \Delta e \right|}{\left| e \right|}\rangle$',fontsize=40)


plt.tick_params(
            top=True,
            right=True,
            labeltop=False,
            labelright=False,
            direction="in",
            labelsize=20,
            width=5,
        )
plt.xticks([1000,5000,10000,15000,20000,25000,30000,60000,80000],[r'$10^3$',r'$5 \cdot 10^3$',r'$10^4$',r'$1.5 \cdot 10^4$',r'$2 \cdot 10^4$',r'$2.5 \cdot 10^4$',r'$3 \cdot 10^4$',r'$6 \cdot 10^4 $',r"$8 \cdot 10^4$"])
plt.yticks([0.05,0.01,0.005,0.001],[r'$5 \cdot 10^{-2}$',r'$10^{-2}$',r'$5 \cdot 10^{-3}$',r'$ 10^{-3}$'])
#plt.axhline(10**-5,color='red',linestyle='--',linewidth=2,label=r'$0.001 \%$ error')
plt.loglog()
plt.legend(fontsize=20)
plt.show()

#### Convergence of $\Delta e/ e$ for different sizes

In [ ]:
fig=plt.figure(figsize=(10,10))
for i,l in enumerate(ls):
    e_av=np.average((min_energy[l]-gs_energy[l])/np.abs(gs_energy[l]),axis=1)
    plt.plot(epochs[i],e_av,label=f'l={l}',linewidth=3)
plt.legend(fontsize=20)
plt.xlabel(r'$t$',fontsize=40)
plt.ylabel(r'$\langle\frac{\Delta e}{ e }\rangle$',fontsize=40)

plt.tick_params(
            top=True,
            right=True,
            labeltop=False,
            labelright=False,
            direction="in",
            labelsize=20,
            width=5,
        )
plt.xticks([0,10000,20000,30000,40000,50000,60000,80000],[r'$0$',r'$1 \cdot 10^4$',r'$2 \cdot 10^4$',r'$3 \cdot 10^4$',r'$4 \cdot 10^4$',r'$5 \cdot 10^4$',r'$6 \cdot 10^4$',r'$8 \cdot 10^4$'])
plt.yticks([0.01,0.005,0.001],[r'$10^{-2}$',r'$5 \cdot 10^{-3}$',r'$ 10^{-3}$'])
plt.axhline(-0.00,color='red',label='zero')
plt.axhline(-0.001,color='red',linestyle='--',label=r'$0.1 \%$ error')
#plt.semilogy()
plt.legend(fontsize=20)
plt.show()

#### Convergence of $ |\Delta z|/ |z| $ for different sizes

In [ ]:
fig=plt.figure(figsize=(10,10))
errors_n=[]
sigma_errors_n=[]
for i,l in enumerate(ls):
    dn_av=np.average((np.average(np.abs(min_density[l]-gs_density[l]),axis=-1)/np.average(np.abs(gs_density[l]),axis=-1)),axis=1)
    plt.plot(epochs[i],dn_av,label=f'l={l}',linewidth=3)
    errors_n.append(dn_av[-1])
    sigma_errors_n.append(np.std((np.average(np.abs(min_density[l]-gs_density[l]),axis=-1)/np.average(np.abs(gs_density[l]),axis=-1)),axis=1)[-1])
plt.legend(fontsize=20)
plt.xlabel(r'$t$',fontsize=40)
plt.ylabel(r'$\langle\frac{\left|\Delta z \right|}{\left| z \right|}\rangle$',fontsize=40)

plt.tick_params(
            top=True,
            right=True,
            labeltop=False,
            labelright=False,
            direction="in",
            labelsize=20,
            width=5,
        )
plt.xticks([0,10000,20000,30000,40000,50000,60000,80000],[r'$0$',r'$1 \cdot 10^4$',r'$2 \cdot 10^4$',r'$3 \cdot 10^4$',r'$4 \cdot 10^4$',r'$5 \cdot 10^4$',r'$6 \cdot 10^4$',r"$8 \cdot 10^4$"])
plt.yticks([0.1,0.05,0.01,0.001],[r"$10^{-1}$",r'$5 \cdot 10^{-2}$',r'$10^{-2}$',r'$ 10^{-3}$'])
plt.axhline(0.001,color='red',label=r'$0.1 \%$ error')
#plt.semilogy()
plt.legend(fontsize=20)
plt.show()

#### Non unique V-representability issue dfs